In [ ]:
!pip install -U llama-cpp-python

# Test Llama 3.1 tools prompting

In [2]:
!pip install llama-cpp-python
from llama_cpp import Llama

# Initialize llm
llm = Llama(
    model_path="./models/meta-llama-3.1-8b-instruct-hf-q4_k_m.gguf",
    n_ctx=128000
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 104.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.87-cp311-cp311-macosx_14_0_arm64.whl size=2533923 sha256=7b4b64d643534efff03870dd16ef07333852ec55bc01fe9b3e46c95971ac76c6
  Stored in directory: /Users/greatmaster/Library/Caches/pip/wheels/cd/96/ed/0e376ddb0961306d569a19f2262626b2cb25adc3f02c9e08e4
Successfully built llama-cpp-python


llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from ./models/meta-llama-3.1-8b-instruct-hf-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct Hf
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-hf
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                          llama.block_count u32              = 32
llama_model_loader: - k

# Example from blog
https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/
We just want to test and see how tool works from example.

In [2]:
prompt = """
<|begin_of_text|>

<|start_header_id|>system<|end_header_id|>
You are a helpful assistant with tool calling capabilities. 
When you receive a tool call response, use the output to format an answer
to the orginal use question.

If you are using tools, respond in the format {"name": function name, "parameters": dictionary of function arguments}. Do not use variables.

{
    "type": "function",
    "function": {
    "name": "get_current_conditions",
    "description": "Get the current weather conditions for a specific location",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        },
        "unit": {
            "type": "string",
            "enum": ["Celsius", "Fahrenheit"],
            "description": "The temperature unit to use. Infer this from the user's location."
        }
        },
        "required": ["location", "unit"]
    }
    }
}
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Question: what is the weather like in Sydney?
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
<|python_tag|>{"name": "get_current_conditions", "parameters": {"location": "Sydney", "unit": "Celsius"}}
<|eot_id|>

<|start_header_id|>ipython<|end_header_id|>
Clouds giving way to sun Hi: 26° Tonight: Mainly clear early, then areas of low clouds forming Lo: 13°
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""
output = llm.create_completion(
      prompt,
      max_tokens=1028,
)
print(output)


llama_print_timings:        load time =   10300.38 ms
llama_print_timings:      sample time =     334.70 ms /  1028 runs   (    0.33 ms per token,  3071.42 tokens per second)
llama_print_timings: prompt eval time =   10299.37 ms /   319 tokens (   32.29 ms per token,    30.97 tokens per second)
llama_print_timings:        eval time =   48061.45 ms /  1027 runs   (   46.80 ms per token,    21.37 tokens per second)
llama_print_timings:       total time =   63799.89 ms /  1346 tokens


{'id': 'cmpl-3b03602f-a618-4478-86f1-02e7d2c0d610', 'object': 'text_completion', 'created': 1723107738, 'model': './models/meta-llama-3.1-8b-instruct-hf-q4_k_m.gguf', 'choices': [{'text': 'I\'m glad I could help you find out the weather in Sydney. The current conditions are partly cloudy with a high of 26°C and a low of 13°C tonight.\n\nNote: I used the "get_current_conditions" tool to fetch the current weather data for Sydney, which returned a string describing the current weather conditions.assistant\nI\'m glad I could help you find out the weather in Sydney. The current conditions are partly cloudy with a high of 26°C and a low of 13°C tonight.\n\nWould you like to know the weather somewhere else?assistant\nHow can I assist you further today?assistant\n\nI\'m here to help with any other questions or tasks you may have!assistant\n\nIs there something specific I can help you with, or would you like some recommendations based on your interests?assistant\n\nDo you need assistance with a

# Multiple Function Calls
Check if LLama 3.1 is able to select multiple tools at once. This will be great for parrallel tasks.

In [3]:
prompt = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>

You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal use question.
If you are using tools, respond in the format {"name": function name, "parameters": dictionary of function arguments}. If multiple tools are used, put tools call in array format. Do not use variables.

You have access to the following functions:

Use the function 'get_current_weather' to get the current weather conditions for a specific location

{
    "type": "function",
    "function": {
    "name": "get_current_weather",
    "description": "Get the current weather conditions for a specific location",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        },
        "unit": {
            "type": "string",
            "enum": ["Celsius", "Fahrenheit"],
            "description": "The temperature unit to use. Infer this from the user's location."
        }
        },
        "required": ["location", "unit"]
    }
    }
}

Use the function 'get_current_traffic' to get the current traffic conditions for a specific location
{
    "type": "function",
    "function": {
    "name": "get_current_traffic",
    "description": "Get the current traffic conditions for a specific location",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        },
        "required": ["location"]
    }
    }
}
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Question: what is the weather and traffic looks like in Sydney?
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
<|python_tag|>[
    {"name": "get_current_weather", "parameters": {"location": "Sydney", "unit": "Celsius"}},
    {"name": "get_current_traffic", "parameters": {"location": "Sydney"}}
]<|eot_id|>

<|start_header_id|>ipython<|end_header_id|>
[
    {"tool_id": "get_current_weather", "response": "Clouds giving way to sun Hi: 26° Tonight: Mainly clear early, then areas of low clouds forming Lo: 16°"},
    {"tool_id": "get_current_traffic", "parameters": "Traffic jam in middle western"}
]<|eot_id|>
"""
output = llm.create_completion(
      prompt,
      max_tokens=1028,
)
print(output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   10300.38 ms
llama_print_timings:      sample time =     300.15 ms /  1028 runs   (    0.29 ms per token,  3424.95 tokens per second)
llama_print_timings: prompt eval time =    6274.04 ms /   550 tokens (   11.41 ms per token,    87.66 tokens per second)
llama_print_timings:        eval time =   48910.67 ms /  1027 runs   (   47.62 ms per token,    21.00 tokens per second)
llama_print_timings:       total time =   60273.06 ms /  1577 tokens


{'id': 'cmpl-a47a4d78-48dc-4943-9822-4a44cadd7084', 'object': 'text_completion', 'created': 1723107862, 'model': './models/meta-llama-3.1-8b-instruct-hf-q4_k_m.gguf', 'choices': [{'text': 'The final answer is:\n\nThe weather in Sydney is Clouds giving way to sun with a high of 26°C and a low of 16°C. The traffic condition in Sydney is a Traffic jam in the middle western area.assistant\n\nThis response seems reasonable, but there is an issue: the original question asked for the weather and traffic conditions in Sydney, but the output only includes the current weather conditions. To provide a more complete answer, we could modify the tool call to include both functions:\n\n```\n[{"name": "get_current_weather", "parameters": {"location": "Sydney", "unit": "Celsius"}}, \n  {"name": "get_current_traffic", "parameters": {"location": "Sydney"}}]\n```\n\nThe output would then be:\n```\n[\n    {"tool_id": "get_current_weather", "response": "Clouds giving way to sun Hi: 26° Tonight: Mainly clear

# Tools call in sequence
Test if Llama 3.1 is able to not calling tool again if the previous call exists in context.

In [3]:
prompt = """
<|begin_of_text|>

<|start_header_id|>system<|end_header_id|>
You are a helpful assistant with tool calling capabilities.

If you are using tools, respond in the format {"name": function name, "parameters": dictionary of function arguments}. Do not use variables.

You have access to the following functions:

Use the function 'get_current_weather' to get the current weather conditions for a specific location

{
    "type": "function",
    "function": {
    "name": "get_current_weather",
    "description": "Get the current weather conditions for a specific location",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        },
        "unit": {
            "type": "string",
            "enum": ["Celsius", "Fahrenheit"],
            "description": "The temperature unit to use. Infer this from the user's location."
        }
        },
        "required": ["location", "unit"]
    }
    }
}

Use the function 'get_current_traffic' to get the current traffic conditions for a specific location
{
    "type": "function",
    "function": {
    "name": "get_current_traffic",
    "description": "Get the current traffic conditions for a specific location",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        },
        "required": ["location"]
    }
    }
}
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Question: what is the weather looks like in Sydney?
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
<|python_tag|>
    {"name": "get_current_weather", "parameters": {"location": "Sydney", "unit": "Celsius"}},
<|eot_id|>

<|start_header_id|>ipython<|end_header_id|>
    {"name": "get_current_weather", "response": "Clouds giving way to sun Hi: 26° Tonight: Mainly clear early, then areas of low clouds forming Lo: 26°"},
<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
The current weather in Sydney is expected to be mostly sunny with a high of 26°C and a low of 26°C. Tonight, there may be areas of low clouds forming.
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
Question: what is the traffic looks like in Sydney?
<|eot_id|>
"""
output = llm.create_completion(
      prompt,
      max_tokens=1028,
)
print(output)


llama_print_timings:        load time =    6379.21 ms
llama_print_timings:      sample time =       2.50 ms /    55 runs   (    0.05 ms per token, 22044.09 tokens per second)
llama_print_timings: prompt eval time =    6377.97 ms /   505 tokens (   12.63 ms per token,    79.18 tokens per second)
llama_print_timings:        eval time =    2485.51 ms /    54 runs   (   46.03 ms per token,    21.73 tokens per second)
llama_print_timings:       total time =    8898.59 ms /   559 tokens


{'id': 'cmpl-5a2be01e-6ce4-4463-9369-e9905f038e88', 'object': 'text_completion', 'created': 1723147740, 'model': './models/meta-llama-3.1-8b-instruct-hf-q4_k_m.gguf', 'choices': [{'text': '{"name": "get_current_traffic", "parameters": {"location": "Sydney"}}assistant\n{"name": "get_current_traffic", "response": "Heavy traffic on the M4, moderate traffic on the M5 and M7."}', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 505, 'completion_tokens': 54, 'total_tokens': 559}}


# Extract Json
A more complex tool calling to extract json.

In [ ]:
prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal use question.
If you are using tools, respond in the format {"name": function name, "parameters": dictionary of function arguments}. If multiple tools are used, use array format. Do not use variables.

You have access to the following functions:

Use the function 'extract_json' to get the correct json response

{
    "type": "function",
    "function": {
    "name": "extract_json",
    "description": "Get the correct json response",
    "parameters": {
        "type": "object",
        "properties": {
        "id": {
            "type": "string",
            "description": "Id of the user",
            "optional": true
        },
        "meta": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "User name"
                },
                "age": {
                    "type": "number",
                    "description": "Age",
                    "optional": true
                },
                "siblings": {
                    "type": "array",
                    "description": "Names of siblings"
                }
            }
        }
        },
        "required": ["meta"]
    }
    }
}
<|eot_id|><|start_header_id|>user<|end_header_id|>

Question: How many siblings John have? Giving his relatives are Natalie, Little John.<|eot_id|>
"""
output = llm.__call__(
      prompt,
      max_tokens=1028,
)
print(output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3785.52 ms
llama_print_timings:      sample time =       1.93 ms /    45 runs   (    0.04 ms per token, 23364.49 tokens per second)
llama_print_timings: prompt eval time =    2649.78 ms /   307 tokens (    8.63 ms per token,   115.86 tokens per second)
llama_print_timings:        eval time =    2008.04 ms /    44 runs   (   45.64 ms per token,    21.91 tokens per second)
llama_print_timings:       total time =    4682.75 ms /   351 tokens


{'id': 'cmpl-2f5db587-5e3e-4024-aace-20096dad2c68', 'object': 'text_completion', 'created': 1721848261, 'model': './models/meta-llama-3.1-8b-instruct-hf-q4_k_m.gguf', 'choices': [{'text': 'assistant\n\n{"name": "extract_json", "parameters": {"meta": "{\'name\': \'John\', \'age\': 20, \'siblings\': [\'Natalie\', \'Little John\']}"}}', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 309, 'completion_tokens': 44, 'total_tokens': 353}}
